# TF-IDF Cost Sensitive Loss

In [1]:
import time
import os
import pyhocon
import torch
import argparse
from torch import nn
from torch import optim
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import glob
import os, re, json
import matplotlib.pyplot as plt
import jsonlines
import random
import torch.nn.functional as F
import math
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')

+ データ処理
    + データのロード
    + データの前処理
    + データの分割
+ モデルの構築
    + Cmb Attentionモデル
    + 損失関数の定義
        + tf-idf形式のコスト損失関数の構築
        
+ モデルの訓練
    + モデルの学習
    + モデルの精度評価

## データ処理

### データのディレクトリを取得

In [3]:
data_url_repo = "data/corpus/*"
data_url_dir = glob.glob(data_url_repo)

In [4]:
EOS_token = '<EOS>'
BOS_token = '<BOS>'
file_pattern = re.compile(r'^sw\_([a-z]+?)\_([0-9]+?)\.jsonlines$')

swda_tagu = {
    '<Uninterpretable>': ['abandoned_or_turn-exit/uninterpretable', 'non-verbal'],
    '<Statement>': ['statement-non-opinion', 'statement-opinion', 'other_answers', '3rd-party-talk', 'self-talk', 'offers,_options_commits', 'collaborative_completion'],
    '<Question>': ['q', 'yes-no-question', 'wh-question', 'declarative_yes-no-question', 'backchannel_in_question_form', 'open-question', 'rhetorical-questions', 'signal-non-understanding', 'or-clause', 'tag-question', 'declarative_wh-question'],
    '<Directive>': ['action-directive'],
    '<Greeting>': ['conventional-opening', 'conventional-closing'],
    '<Apology>': ['apology', 'no_answers', 'reject', 'negative_non-no_answers', 'dispreferred_answers', 'dispreferred_answers'],
    '<Agreement>': ['agree/accept', 'maybe/accept-part', 'thanking'],
    '<Understanding>': ['acknowledge_(backchannel)', 'summarize/reformulate', 'appreciation', 'response_acknowledgement', 'affirmative_non-yes_answers', 'yes_answers'],
    '<Other>': ['other', 'hedge', 'quotation', 'repeat-phrase', 'hold_before_answer/agreement', 'downplayer']
}

daily_tagu = {1: "inform", 2: "question", 3: "directive", 4: "commissive"}

### 対話行為のID化

In [5]:
class DA_to_ID:
    
    def __init__(self, config, X_DA, Y_DA, name):
        self.word2id = None
        self.id2word = None
        self.config = config
        self.X_DA = X_DA
        self.Y_DA = Y_DA
        self.name = name
        self.construct()
        
    def construct(self):
#        vocab = {'<PAD>': 0}
        vocab = {}
        vocab_count = {}
        
        for x,y in zip(self.X_DA, self.Y_DA):
            for token in x:
                if token in vocab_count:
                    vocab_count[token] += 1
                else:
                    vocab_count[token] = 1
                    
            for token in y:
                if token in vocab_count:
                    vocab_count[token] += 1
                else:
                    vocab_count[token] = 1
                    
        for k, _ in sorted(vocab_count.items(), key=lambda x: -x[1]):
            vocab[k] = len(vocab)
            if len(vocab) >= self.config[self.name]['MAX_VOCAB']: break
        self.word2id = vocab
        self.id2word = {v : k for k, v in vocab.items()}
        return vocab
        
    def tokenize(self, X_tensor, Y_tensor):
        X_Tensor = [[self.word2id[token] for token in sentence] for sentence in X_tensor]
        Y_Tensor = [[self.word2id[token] for token in sentence] for sentence in Y_tensor]
        return X_Tensor, Y_Tensor

### 発話のID化

In [6]:
class UTT_to_ID:
    
    def __init__(self, config, X_UTT, Y_UTT, name):
        self.word2id = None
        self.id2word = None
        self.config = config
        self.X_UTT = X_UTT
        self.Y_UTT = Y_UTT
        self.name = name
        self.construct()
        
    def construct(self):
        
        vocab = {'<UNK>': 0, '<EOS>': 1, '<BOS>': 2, '<UttPAD>': 3, '<ConvPAD>': 4}
        vocab_count = {}
        
        for x,y in zip(self.X_UTT, self.Y_UTT):
            for seq in x:
                for word in seq:
                    if word in vocab_count:
                        vocab_count[word] += 1
                    else:
                        vocab_count[word] = 1
            for seq in y:
                for word in seq:
                    if word in vocab_count:
                        vocab_count[word] += 1
                    else:
                        vocab_count[word] = 1
                        
        for k, _ in sorted(vocab_count.items(), key=lambda x: -x[1]):
            vocab[k] = len(vocab)
            if len(vocab) >= self.config[self.name]['UTT_MAX_VOCAB']: break
        self.word2id = vocab
        self.id2word = {v : k for k, v in vocab.items()}

        return vocab
        
    def tokenize(self, X_tensor, Y_tensor):
        
        X_Tensor = [[[self.word2id[token] if token in self.word2id else self.word2id['<UNK>'] for token in seq] for seq in dialogue] for dialogue in X_tensor]
        Y_Tensor = [[[self.word2id[token] if token in self.word2id else self.word2id['<UNK>'] for token in seq] for seq in dialogue] for dialogue in Y_tensor]
        return X_Tensor, Y_Tensor

### トレーニングデータ作成

In [7]:
def create_traindata(config, name):
    files = [f for f in os.listdir(config[name]['train_path']) if file_pattern.match(f)]
    # print("files:" , files)
    da_x, da_y, utt_x, utt_y, turn = [], [], [], [], []
    # 1file 1conversation
    for filename in files:
        # print(os.path.join(config['train_path'], filename))
        with open(os.path.join(config[name]['train_path'], filename), 'r') as f:
            data = f.read().split('\n')
            data.remove('')
            da_seq, utt_seq, turn_seq = [], [], []
            # 1line 1turn
            for idx, line in enumerate(data, 1):
                jsondata = json.loads(line)
                # single-turn multi dialogue case
                if config[name]['multi_dialogue']:
                    for da, utt in zip(jsondata['DA'], jsondata['sentence']):
                        da_seq.append(da)
                        utt_seq.append(utt.split(' '))
                        turn_seq.append(0)
                    if config[name]['turn']:
                        da_seq.append('<turn>')
                        utt_seq.append('<turn>')
                    turn_seq[-1] = 1
                # single-turn single dialogue case
                else:
                    da_seq.append(jsondata['DA'][-1])
                    utt_seq.append(jsondata['sentence'][-1].split(' '))
            da_seq = [easy_damsl(da) for da in da_seq]
        
            
            # assert len(turn_seq) == len(da_seq), '{} != {}'.format(len(turn_seq), len(da_seq))
        if config[name]['state']:
            for i in range(max(1, len(da_seq) - 1 - config[name]['window_size'])):
                da_x.append(da_seq[i:min(len(da_seq)-1, i + config[name]['window_size'])])
                da_y.append(da_seq[1 + i:min(len(da_seq), 1 + i + config[name]['window_size'])])
                utt_x.append(utt_seq[i:min(len(da_seq)-1, i + config[name]['window_size'])])
                utt_y.append(utt_seq[1 + i:min(len(da_seq), 1 + i + config[name]['window_size'])])
                turn.append(turn_seq[i:min(len(da_seq), i + config[name]['window_size'])])
        else:
            da_x.append(da_seq[:-1])
            da_y.append(da_seq[1:])
            utt_x.append(utt_seq[:-1])
            utt_y.append(utt_seq[1:])
            turn.append(turn_seq[:-1])
    assert len(da_x) == len(da_y), 'Unexpect length da_posts and da_cmnts'
    assert len(utt_x) == len(utt_y), 'Unexpect length utt_posts and utt_cmnts'
    # assert len(turn) == len(da_posts)
    
    return da_x, da_y, utt_x, utt_y, turn


### タグ

In [8]:
def easy_damsl(tag):
    easy_tag = [k for k, v in swda_tagu.items() if tag in v]
    return easy_tag[0] if not len(easy_tag) < 1 else tag

### データの分割

In [9]:
def separate_data(x, y, turn):
    split_size = round(len(x) / 10)
    if split_size == 0: split_size = 1
    X_train, Y_train, Tturn = x[split_size * 2:], y[split_size * 2:], turn[split_size * 2:]
    X_valid, Y_valid, Vturn = x[split_size: split_size * 2], y[split_size: split_size * 2], turn[split_size: split_size * 2]
    X_test, Y_test, Testturn = x[:split_size], y[:split_size], turn[:split_size]
    assert len(X_train) == len(Y_train), 'Unexpect to separate train data'
    return X_train, Y_train, X_valid, Y_valid, X_test, Y_test, Tturn, Vturn, Testturn

### 訓練の準備

In [10]:
def initialize_env(name):
    config = pyhocon.ConfigFactory.parse_file('./dialogue.conf')
    config['log_dirs'] = os.path.join(config[name]['log_dir'])
    if not os.path.exists(config['log_dirs']):
        os.mkdir(config['log_dirs'])
     
    return config

In [11]:
def create_DAdata(config, name):
    posts, cmnts, _, _, turn = create_traindata(config, name)
    X_train, Y_train, X_valid, Y_valid, X_test, Y_test, Tturn, Vturn, Testturn = separate_data(posts, cmnts, turn)
    return X_train, Y_train, X_valid, Y_valid, X_test, Y_test, Tturn, Vturn, Testturn

def create_Uttdata(config, name):
    _, _, posts, cmnts, turn = create_traindata(config, name)
    X_train, Y_train, X_valid, Y_valid, X_test, Y_test, _, _, _ = separate_data(posts, cmnts, turn)
    return X_train, Y_train, X_valid, Y_valid, X_test, Y_test


In [13]:
write = SummaryWriter("./logs")

model_name = 'CmbAttention'
# loss_name = "Affinity"
loss_name = "CE"
# loss_name = "TFIDF"
config = initialize_env(model_name+loss_name)

XDA_train, YDA_train, XDA_valid, YDA_valid, _, _, Tturn, Vturn, _ = create_DAdata(config, model_name+loss_name)
XUtt_train, YUtt_train, XUtt_valid, YUtt_valid, _, _ = create_Uttdata(config, model_name+loss_name)

DA_vocab = DA_to_ID(config, XDA_train+XDA_valid, YDA_train+YDA_valid, model_name+loss_name)
Utt_vocab = UTT_to_ID(config, XUtt_train+XUtt_valid, YUtt_train+YUtt_valid, model_name+loss_name)

XDA_train, YDA_train = DA_vocab.tokenize(XDA_train, YDA_train)
XDA_valid, YDA_valid = DA_vocab.tokenize(XDA_valid, YDA_valid)
XUtt_train, YUtt_train = Utt_vocab.tokenize(XUtt_train, YUtt_train)
XUtt_valid, YUtt_valid = Utt_vocab.tokenize(XUtt_valid, YUtt_valid)

print('Finish preparing dataset...')

Finish preparing dataset...


## モデルの構築

#### Cmb Attention

In [14]:
class CmbAttentionModel(nn.Module):
    
    def __init__(self, model_name, utt_vocab, da_vocab, config, device):
        super(CmbAttentionModel, self).__init__()
        
        self.utter_encoder = UtteraceEncoder(len(utt_vocab.word2id), config[model_name]['UTT_EMBED'], config[model_name]['UTT_HIDDEN'])

        self.context_encoder = RNNContextAwareEncoder(config[model_name]['CON_EMBED'], config[model_name]['CON_HIDDEN'])

        self.da_encoder = RNNDAAwareEncoder(len(utt_vocab.word2id), config[model_name]['DA_EMBED'], config[model_name]['DA_HIDDEN'])

        self.de_encoder = DenceEncoder(config[model_name]['DA_HIDDEN'] + config[model_name]['CON_HIDDEN'], config[model_name]['DA_EMBED'], len(da_vocab.word2id))

        self.device = device
        
    def forward(self, X_utter, X_da, Y_da, mask, utter_hidden, context_hidden, da_hidden, turn):

        utter_output, utter_weights = self.utter_encoder(X_utter, mask)

        turn_output = torch.cat((utter_output, turn), dim=2)

        context_output, context_weights, context_hidden = self.context_encoder(turn_output, mask, context_hidden)

        da_output, da_weights, da_hidden = self.da_encoder(X_da, mask, da_hidden)

        x_output = torch.cat((context_output, da_output), dim=2)

        dence_output = self.de_encoder(x_output)

        output = dence_output.squeeze(1)  # (batch_size, da_dim)
        
        Y_da = Y_da.squeeze()
        
        return output, utter_hidden, context_hidden, da_hidden
    
    def validtion(self, X_utter, X_da, Y_da, mask, utter_hidden, context_hidden, da_hidden, turn):

        utter_output, utter_weights = self.utter_encoder(X_utter, mask)

        turn_output = torch.cat((utter_output, turn), dim=2)

        context_output, context_weights, context_hidden = self.context_encoder(turn_output, mask, context_hidden)

        da_output, da_weights, da_hidden = self.da_encoder(X_da, mask, da_hidden)

        x_output = torch.cat((context_output, da_output), dim=2)

        dence_output = self.de_encoder(x_output)

        output = dence_output.squeeze(1)  # (batch_size, da_dim)
            
        Y_da = Y_da.squeeze(0)

        return output, utter_hidden, context_hidden, da_hidden, Y_da


    def prediction(self, X_utter, X_da, mask, utter_hidden, context_hidden, da_hidden, turn):

        utter_output, utter_weights = self.utter_encoder(X_utter, mask)

        turn_output = torch.cat((utter_output, turn), dim=2)

        context_output, context_weights, context_hidden = self.context_encoder(turn_output, mask, context_hidden)

        da_output, da_weights, da_hidden = self.da_encoder(X_da, mask, da_hidden)

        x_output = torch.cat((context_output, da_output), dim=2)

        dence_output = self.de_encoder(x_output)

        output = dence_output.squeeze(1)  # (batch_size, da_dim)

        return output, utter_hidden, context_hidden, da_hidden, utter_weights


    def initDAHidden(self, batch_size):
        return self.utter_encoder.initHidden(batch_size, self.device), self.context_encoder.initHidden(batch_size, self.device), self.da_encoder.initHidden(batch_size, self.device)

#### Model Block

In [15]:
class WordEmbedding(nn.Module):

    def __init__(self, vocab_size, embed_size, w_model):
        super(WordEmbedding, self).__init__()
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.linear = nn.Linear(embed_size, w_model)

    def forward(self, x_word):
        return torch.tanh(self.linear(self.word_embedding(x_word)))

In [16]:
class DAEmbedding(nn.Module):

    def __init__(self, da_size, embed_size, d_model):
        super(DAEmbedding, self).__init__()
        self.da_embedding = nn.Embedding(da_size, embed_size)
        self.linear = nn.Linear(embed_size, d_model)

    def forward(self, x_da):
        return torch.tanh(self.linear(self.da_embedding(x_da)))

In [17]:
class Attention(nn.Module):

    def __init__(self, d_model):
        super(Attention, self).__init__()
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, d_model)
        self.d_k = d_model
       
    def forward(self, q, k, v, mask=None):
        # 全結合層で特徴量を変換
        k = self.k_linear(k)
        q = self.q_linear(q)
        v = self.v_linear(v)

        # Attentionの値を計算する
        # 各値を足し算すると大きくなりすぎるので、root(d_k)で割って調整
        weights = torch.matmul(q, k.transpose(1, 2)) / math.sqrt(self.d_k)
        
        # ここでmaskを計算
        if mask is not None:
            mask = mask.unsqueeze(1)
            weights = weights.masked_fill(mask == 0, -1e9)

        # softmaxで規格化をする
        attention_weights = F.softmax(weights, dim=-1)

        # AttentionをValueとかけ算
        output = torch.matmul(attention_weights, v)

        # 全結合層で特徴量を変換
        output = self.out(output)

        return output, attention_weights

In [18]:
class ContextAttention(nn.Module):

    def __init__(self, d_model, hidden_size, att_size):
        super(ContextAttention, self).__init__()
        self.q_linear = nn.Linear(att_size, att_size)
        self.v_linear = nn.Linear(att_size, att_size)
        self.k_linear = nn.Linear(att_size, att_size)

        self.fc_1 = nn.Linear(d_model, d_model)
        self.fc_3 = nn.Linear(hidden_size, d_model, bias=True)
        self.fc_2 = nn.Linear(d_model, att_size)

        self.fc_out = nn.Linear(att_size, hidden_size, bias=True)
        self.d_k = att_size

    def forward(self, x, mask, hidden):
        
        x = self.fc_2(torch.tanh(self.fc_1(x) + self.fc_3(hidden)))

        q = self.q_linear(x)
        v = self.v_linear(x)
        k = self.k_linear(x)

        weights = torch.matmul(q, k.transpose(1, 2)) / math.sqrt(self.d_k)

        # ここでmaskを計算
        if mask is not None:
            mask = mask.unsqueeze(1)
            weights = weights.masked_fill(mask == 0, -1e9)

        attention_weights = F.softmax(weights, dim=-1)

        att_output = torch.matmul(attention_weights, v)

        output = self.fc_out(att_output)

        return output, attention_weights

In [19]:
class FeedForward(nn.Module):

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForward, self).__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):

        x = self.linear_1(x)

        x = self.dropout(F.relu(x))

        x = self.linear_2(x)

        return x

In [20]:
class PositinalEncoding(nn.Module):

    def __init__(self, d_model, max_len, dropout=0.1):
        super(PositinalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        
        x = x + self.pe[:x.size(0), :]

        return self.dropout(x)


In [21]:
class DenceEncoder(nn.Module):
    def __init__(self, da_hidden, da_embed_size, da_input_size):
        super(DenceEncoder, self).__init__()
        self.he = nn.Linear(da_hidden, da_embed_size)
        self.ey = nn.Linear(da_embed_size, da_input_size)

    def forward(self, hidden):
        pred = self.ey(torch.tanh(self.he(hidden)))
        return pred


In [22]:
class RNNContextAwareEncoder(nn.Module):
    
    def __init__(self, emb_dim, d_model):
        super(RNNContextAwareEncoder, self).__init__()
        self.d_model = d_model
        self.linear = nn.Linear(self.d_model+1, self.d_model)
        self.rnn = nn.GRU(self.d_model, self.d_model, batch_first=True)
        self.attention = ContextAttention(self.d_model, self.d_model, self.d_model)
        self.ffn = FeedForward(self.d_model, self.d_model)

    def forward(self, x, mask, hidden):

        lin_output = self.linear(x)

        att_output, att_weights = self.attention(lin_output, mask, hidden.transpose(0,1))        

        rnn_output, rnn_hidden = self.rnn(att_output, hidden)

        ffn_output = self.ffn(rnn_output)

        return ffn_output, att_weights, rnn_hidden

    def initHidden(self, batch_size, device):
        return torch.zeros(1, batch_size, self.d_model).to(device)

In [23]:
class RNNDAAwareEncoder(nn.Module):

    def __init__(self, da_size, emb_dim, d_model):
        super(RNNDAAwareEncoder, self).__init__()
        self.d_model = d_model
        self.embedding = DAEmbedding(da_size, emb_dim, self.d_model)
        self.rnn = nn.GRU(self.d_model, self.d_model, batch_first=True)
        # self.attention = ContextAwareAttention(self.d_model, self.d_model, self.d_model)
        self.attention = ContextAttention(self.d_model, self.d_model, self.d_model)
        self.ffn = FeedForward(self.d_model, emb_dim)

    def forward(self, X_da, mask, hidden):

        emb_output = self.embedding(X_da)

        att_output, att_weights = self.attention(emb_output, mask, hidden.transpose(0,1))        

        rnn_output, rnn_hidden = self.rnn(att_output, hidden)

        ffn_output = self.ffn(rnn_output)

        return ffn_output, att_weights, rnn_hidden

    def initHidden(self, batch_size, device):
        return torch.zeros(1, batch_size, self.d_model).to(device)

In [24]:
class UtteraceEncoder(nn.Module):
    
    def __init__(self, vocab_size, emb_dim, d_model):
        super(UtteraceEncoder, self).__init__()
        self.d_model = d_model
        self.embedding = WordEmbedding(vocab_size, emb_dim, self.d_model)
        self.pe = PositinalEncoding(self.d_model, 200)
        self.att = Attention(self.d_model)
        self.ffn = FeedForward(d_model, emb_dim)
        
    def forward(self, x_utter, mask):

        emb_output = self.embedding(x_utter)

        pos_output = self.pe(emb_output)

        att_output, att_weights = self.att(pos_output, pos_output, pos_output, mask)

        ffn_output = self.ffn(att_output)

        seq_len = ffn_output.size()[1]

        avg_output = F.avg_pool2d(ffn_output, (seq_len, 1)) # => (128, 1, 512)

        return avg_output, att_weights  # 発話ベクトル(128, 1, 512)

    def initHidden(self, batch_size, device):
        return torch.zeros(1, batch_size, self.d_model).to(device)

### TF-IDF計算

In [25]:
def ClacTF(DA_num, dialogue_len):
    return DA_num/dialogue_len

In [26]:
def ClacIDF(sum_num, dialogue_num):
    return np.log(sum_num/(dialogue_num+1))

In [27]:
classes_weight=[1]*9
count_dialogue=[0]*9
for i in range(9):
    for j in range(len(XDA_train)):
        if i in XDA_train[j]:
            count_dialogue[i] += 1
#         count_num[i] += XDA_train[j].count(i)

In [28]:
means_num = [0]*9
for i in range(9):
    for j in XDA_train:
        if i in j:
            means_num[i]+=1
            classes_weight[i] += ClacIDF(len(XDA_train), count_dialogue[i])*ClacTF(j.count(i), len(j))
            
            
for i in range(9):
    classes_weight[i]/=means_num[i]

In [29]:
class TFIDFLoss(nn.Module):
    
    def __init__(self, dialogue_size, batch_size):
        super(TFIDFLoss, self).__init__()
        self.dialogue_size = dialogue_size
        self.batch_size = batch_size
        
    def forward(self, output, target):
        return 
        
    def clacTF(DA_num):
        return DA_num/self.dialogue_size
    
    def clacIDF(dialogue_num):
        return np.log(self.batch_size/(dialogue_num+1))

In [30]:
class_weights=torch.tensor(classes_weight, dtype=torch.float32)

## モデルの訓練

In [31]:
def select_model(mode_name, utt_vocab, da_vocab, config, device, lr, loss_func):
    
    if model_name == "CmbAttention":
        
        if loss_func == "CE":
            model = CmbAttentionModel(model_name+loss_func, utt_vocab, da_vocab, config, device).to(device)
            opt = optim.Adam(model.parameters(), lr)
            loss_function = nn.CrossEntropyLoss(reduction='mean')
        
        elif loss_func == "Affinity":
            model = CmbAttentionModel_affinity(model_name+loss_func, utt_vocab, da_vocab, config, device).to(device)
            opt = optim.Adam(model.parameters(), lr)
            loss_function = Affinity_Loss(0.95, len(da_vocab.word2id), device)
            
        elif loss_func == "TFIDF":
            model = CmbAttentionModel(model_name+loss_func, utt_vocab, da_vocab, config, device).to(device)
            opt = optim.Adam(model.parameters(), lr)
            loss_function = nn.CrossEntropyLoss(reduction='mean', weight=class_weights).to(device)
            
    else:
        model = None
        opt = None
        loss_function = None
        
    return model, opt, loss_function

In [34]:
lr = config[model_name+loss_name]['lr']
batch_size = config[model_name+loss_name]['BATCH_SIZE']
plot_train_losses = []
plot_valid_losses = []
# loss_name = "Affinity"
# loss_name = "CE"
# loss_name = "TFIDF"
    
print_total_loss = 0
plot_total_loss = 0
plot_total_acc = 0
_valid_loss = None

models, optims, loss_func = select_model(model_name, Utt_vocab, DA_vocab, config, device, lr, loss_name)   

#### 検証

In [37]:
@torch.no_grad()
def validation(X_valid, Y_valid, XU_valid, YU_valid, model, device, config, turn):

    total_loss = 0
    idx = 0
    
    for seq_idx in range(len(X_valid)):
        print('\r{}/{} conversation evaluating'.format(seq_idx+1, len(X_valid)), end='')
        utter_hidden, context_hidden, da_hidden = model.initDAHidden(1)
        X_seq = X_valid[seq_idx]
        Y_seq = Y_valid[seq_idx]
        turn[seq_idx] = turn[seq_idx] + [0] * (len(X_seq) - len(turn[seq_idx]))
        turn_seq = turn[seq_idx]
        XU_seq = XU_valid[seq_idx]
        YU_seq = YU_valid[seq_idx]

        assert len(X_seq) == len(Y_seq), 'Unexpect sequence len in evaluate {} != {}'.format(len(X_seq), len(Y_seq))
        
        for i in range(0, len(X_seq)):
            X_tensor = torch.tensor([[X_seq[i]]]).to(device)
            Y_tensor = torch.tensor([[Y_seq[i]]]).to(device)
            turn_tensor = torch.tensor([[turn_seq[i]]]).to(device)
            turn_tensor = turn_tensor.float()
            turn_tensor = turn_tensor.unsqueeze(1)   
            XU_tensor = torch.tensor([XU_seq[i]]).to(device)
            
            output, utter_hidden, context_hidden, da_hidden, Y_tensor = model.validtion(XU_tensor, X_tensor, Y_tensor, None, utter_hidden, context_hidden, da_hidden, turn_tensor)
            loss = loss_func(output, Y_tensor)
            total_loss += loss.item()
            
        total_loss/=5

    return total_loss

#### 訓練

In [ ]:
print("start TRAINING")

start = time.time()

config[model_name+loss_name]['EPOCH'] = 30

for epoch in range(config[model_name+loss_name]['EPOCH']):
    
    tmp_time = time.time()
    print('Epoch {} start'.format(epoch+1))
    index = [i for i in range(len(XDA_train))]
    random.shuffle(index)
    idx=0
    
    while idx < len(index):
        step_size = min(batch_size, len(index)-idx)
        batch_idx = index[idx:idx+step_size]
        utter_hidden, context_hidden, da_hidden = models.initDAHidden(step_size)
        
        print('\rConversation {}/{} training...'.format(idx + step_size, len(XDA_train)), end='')
        Xda_seq = [XDA_train[seq_idx] for seq_idx in batch_idx]
        Yda_seq = [YDA_train[seq_idx] for seq_idx in batch_idx]
        turn_seq = [Tturn[seq_idx] for seq_idx in batch_idx]
        max_conv_len = max(len(s) for s in Xda_seq) 
        
        ## 
        XUtt_seq = [XUtt_train[seq_idx] for seq_idx in batch_idx]
        YUtt_seq = [YUtt_train[seq_idx] for seq_idx in batch_idx]
            
        for i in range(len(XUtt_seq)):
            XUtt_seq[i] = XUtt_seq[i] + [[Utt_vocab.word2id['<ConvPAD>']]] * (max_conv_len - len(XUtt_seq[i]))
            YUtt_seq[i] = YUtt_seq[i] + [[Utt_vocab.word2id['<ConvPAD>']]] * (max_conv_len - len(YUtt_seq[i]))
                
        for ci in range(len(Xda_seq)):

            turn_seq[ci] = turn_seq[ci] + [0] * (max_conv_len - len(turn_seq[ci]))
            Xda_seq[ci] = Xda_seq[ci] + [-1] * (max_conv_len - len(Xda_seq[ci]))
            Yda_seq[ci] = Yda_seq[ci] + [-1] * (max_conv_len - len(Yda_seq[ci]))
            
        for i in range(0, max_conv_len):
            last = True if i == max_conv_len - 1 else False
            
            Xda_tensor = torch.tensor([[X[i]] for X in Xda_seq]).to(device)
            Yda_tensor = torch.tensor([[Y[i]] for Y in Yda_seq]).to(device)
            turn_tensor = torch.tensor([[t[i]] for t in turn_seq]).to(device)
            turn_tensor = turn_tensor.float()
            turn_tensor = turn_tensor.unsqueeze(1)    
                
            max_seq_len = max(len(XU[i]) + 1 for XU in XUtt_seq)
            for ci in range(len(XUtt_seq)):
                XUtt_seq[ci][i] = XUtt_seq[ci][i] + [Utt_vocab.word2id['<UttPAD>']] * (max_seq_len - len(XUtt_seq[ci][i]))
                YUtt_seq[ci][i] = YUtt_seq[ci][i] + [Utt_vocab.word2id['<UttPAD>']] * (max_seq_len - len(YUtt_seq[ci][i]))
            XUtt_tensor = torch.tensor([XU[i] for XU in XUtt_seq]).to(device)
            YUtt_tensor = None

            output, utter_hidden, context_hidden, da_hidden = models(XUtt_tensor, Xda_tensor, Yda_tensor, None, utter_hidden, context_hidden, da_hidden, turn_tensor)
            Yda_tensor = Yda_tensor.squeeze(1)
            loss = loss_func(output, Yda_tensor)
            optims.zero_grad()
            loss.backward(retain_graph=True)
            print_total_loss += loss.item()
                
            if last:
                optims.step()
        print_total_loss/=max_conv_len      
        idx += step_size
        
    valid_loss = validation(XDA_valid, YDA_valid, XUtt_valid, YUtt_valid, models, device, config, Vturn)
    
    
    def save_model(filename):
        torch.save(models.state_dict(), os.path.join(config[model_name+loss_name]['log_dir'], config[model_name+loss_name]['SAVE_NAME'] + "_" + str(config[model_name+loss_name]['window_size']) + ".model".format(filename)))
    
    print_total_loss/(max_conv_len*idx)
    print("steps %d\tloss %.4f\tvalid loss %.4f | exec time %.4f" % (epoch+1, print_total_loss, valid_loss, time.time()-tmp_time))
    print_total_loss = 0
    
    if _valid_loss == None:
        save_model("model_save")
        print("Model Saved")
        _valid_loss = valid_loss
    else:
        if valid_loss<_valid_loss:
            _valid_loss = valid_loss
            save_model("model_save")
            print("Model Saved")
            

start TRAINING
Epoch 1 start
20004/20004 conversation evaluatingsteps 1	loss 1.2765	valid loss 1.9334 | exec time 556.3386
Model Saved
Epoch 2 start
20004/20004 conversation evaluatingsteps 2	loss 1.4217	valid loss 1.7572 | exec time 622.0164
Model Saved
Epoch 3 start
586/20004 conversation evaluatingng...

### モデル評価

In [ ]:
_, _, _, _, XDA_test, YDA_test, _, _, DAturn = create_DAdata(config, model_name)
_, _, _, _, XUtt_test, YUtt_test = create_Uttdata(config, model_name)

XDA_test, YDA_test = DA_vocab.tokenize(XDA_test, YDA_test)
XUtt_test, _ = Utt_vocab.tokenize(XUtt_test, YUtt_test)

print('tokenizer finish')

In [ ]:
models, _, _ = select_model(model_name, Utt_vocab, DA_vocab, config, device, lr, loss_name)
models.load_state_dict(torch.load(os.path.join(config[model_name+loss_name]['log_dir'], config[model_name+loss_name]['SAVE_NAME'] + "_" + str(config[model_name+loss_name]['window_size']) + ".model".format('model_save'))))

print('model loaded')

In [ ]:
@torch.no_grad()

def ModelTest():
    result=[]
    for seq_idx in range(0, len(XDA_test)):
        print('\r{}/{} conversation evaluating'.format(seq_idx+1, len(XDA_test)), end='')
        XDA_seq = XDA_test[seq_idx]
        YDA_seq = YDA_test[seq_idx]
        DAturn[seq_idx] = DAturn[seq_idx] + [0] * (len(XDA_seq) - len(DAturn[seq_idx]))
        DAturn_seq = DAturn[seq_idx]
        XUtt_seq = XUtt_test[seq_idx]

        pred_seq = []
        true_seq = []
        utter_hidden, context_hidden, da_hidden = models.initDAHidden(1)

        for i in range(0, len(XDA_seq)):
            XDA_tensor = torch.tensor([[XDA_seq[i]]]).to(device)
            YDA_tensor = torch.tensor(YDA_seq[i]).to(device)
            DAturn_tensor = torch.tensor([[DAturn_seq[i]]]).to(device)
            DAturn_tensor = DAturn_tensor.float()
            DAturn_tensor = DAturn_tensor.unsqueeze(1)
            XUtt_tensor = torch.tensor([XUtt_seq[i]]).to(device)

            output, utter_hidden, context_hidden, da_hidden, att_weights = models.prediction(XUtt_tensor, XDA_tensor, None, utter_hidden, context_hidden, da_hidden, DAturn_tensor)
                # output, utter_hidden, context_hidden, da_hidden = da_predict_model.prediction(XU_tensor, X_tensor, None, utter_hidden, context_hidden, da_hidden, turn_tensor)

            pred_idx = torch.argmax(output)
            pred_seq.append(pred_idx.item())
            true_seq.append(YDA_tensor.item())
            utter_list = [Utt_vocab.id2word[word] for word in XUtt_seq[i]]

        result.append({'true': true_seq,
                        'true_detok': [DA_vocab.id2word[token] for token in true_seq],
                        'pred': pred_seq,
                        'pred_detok': [DA_vocab.id2word[token] for token in pred_seq],
                        'UttSeq': [[Utt_vocab.id2word[word] for word in sentence] for sentence in XUtt_seq],
                        'seq_detok': [DA_vocab.id2word[label] for label in XDA_seq]})
    return result

In [ ]:
def calc_average(y_true, y_pred):
    p = precision_score(y_true=y_true, y_pred=y_pred, average='macro')
    r = recall_score(y_true=y_true, y_pred=y_pred, average='macro')
    f = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    acc = accuracy_score(y_true=y_true, y_pred=y_pred)
    print('p: {} | r: {} | f: {} | acc: {}'.format(p, r, f, acc))

In [ ]:
result = ModelTest()

In [ ]:
true = [label for line in result for label in line['true']]
pred = [label for line in result for label in line['pred']]

calc_average(y_true=true, y_pred=pred)
f = f1_score(y_true=true, y_pred=pred, average=None)
r = recall_score(y_true=true, y_pred=pred, average=None)
p = precision_score(y_true=true, y_pred=pred, average=None)

p: 0.08928320438668537 | r: 0.2204979571885202 | f: 0.08984448174391911 | acc: 0.14053189362127574


p: 0.08912725942812572 | r: 0.22063442894175095 | f: 0.08982748395954142 | acc: 0.14128174365126975

p: 0.08920915274971167 | r: 0.22057195853621478 | f: 0.08991179627013274 | acc: 0.1412617476504699

#### 2回目の調査結果

+ p: 0.09872020361658407 | r: 0.24682752100587319 | f: 0.1299960965830519 | acc: 0.20530893821235752
               
+ p: 0.09936403344872773 | r: 0.24819802586828738 | f: 0.13074154261395307 | acc: 0.20596880623875224

+ p: 0.0991076895807943 | r: 0.2471875496588724 | f: 0.13046067696477437 | acc: 0.20617876424715056

In [ ]:
inputs = torch.randn(128, 9, requires_grad=True)
print(inputs.size())
target = torch.empty(128, dtype=torch.long).random_(5)
print(target.size())

In [ ]:
inputs, target

In [ ]:
loss = nn.CrossEntropyLoss()
output = loss(inputs, target)
print(output)

In [ ]:
output.item()